In [5]:
import langchain
from dotenv import load_dotenv
load_dotenv()
import os
os.environ["GROQ_API_KEY"]=os.getenv("GROQ_API_KEY")
groq_api_key = os.getenv("GROQ_API_KEY")

In [6]:
from langchain_groq import ChatGroq
model = ChatGroq(model = "gemma2-9b-It", groq_api_key = groq_api_key)
model

ChatGroq(client=<groq.resources.chat.completions.Completions object at 0x000002119D083B80>, async_client=<groq.resources.chat.completions.AsyncCompletions object at 0x000002119D0C4790>, model_name='gemma2-9b-It', model_kwargs={}, groq_api_key=SecretStr('**********'))

In [14]:
from langchain_core.messages import HumanMessage,SystemMessage
# messages : for telling how LLM should behave
messages = [
    SystemMessage(content = "Only Translate the following from English to French"),
    HumanMessage(content = "Hello How are you?")
]
model.invoke(messages)

AIMessage(content='Bonjour, comment allez-vous ? \n', additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 11, 'prompt_tokens': 22, 'total_tokens': 33, 'completion_time': 0.02, 'prompt_time': 0.00013841, 'queue_time': 0.02007851, 'total_time': 0.02013841}, 'model_name': 'gemma2-9b-It', 'system_fingerprint': 'fp_10c08bf97d', 'finish_reason': 'stop', 'logprobs': None}, id='run-24bc53af-4745-453f-8987-a49b2dc8e964-0', usage_metadata={'input_tokens': 22, 'output_tokens': 11, 'total_tokens': 33})

In [15]:
# To get the translation alone
result = model.invoke(messages)
result

AIMessage(content='Bonjour, comment allez-vous ? \n', additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 11, 'prompt_tokens': 22, 'total_tokens': 33, 'completion_time': 0.02, 'prompt_time': 0.000144469, 'queue_time': 0.020464038, 'total_time': 0.020144469}, 'model_name': 'gemma2-9b-It', 'system_fingerprint': 'fp_10c08bf97d', 'finish_reason': 'stop', 'logprobs': None}, id='run-06f33861-5ef5-4e58-8863-83ad7227f92a-0', usage_metadata={'input_tokens': 22, 'output_tokens': 11, 'total_tokens': 33})

In [16]:
# to define the format of the output
from langchain_core.output_parsers import StrOutputParser
parser = StrOutputParser()
parser.invoke(result)

'Bonjour, comment allez-vous ? \n'

LCEL LangChain Expression Language 
To chain components together

In [17]:
# two components 
# 1. model
# 2. string output parser
# Using LCEL we can chain the components together
chain = model | parser
chain.invoke(messages)

'Bonjour, Comment allez-vous ? \n'

In [19]:
from langchain_core.prompts import ChatPromptTemplate

generic_template = 'Translate the following into {language}:'
prompt = ChatPromptTemplate.from_messages(
    [
        ("system",generic_template),
        ("user", "{text}")
    ]
)
prompt

ChatPromptTemplate(input_variables=['language', 'text'], input_types={}, partial_variables={}, messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=['language'], input_types={}, partial_variables={}, template='Translate the following into {language}:'), additional_kwargs={}), HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['text'], input_types={}, partial_variables={}, template='{text}'), additional_kwargs={})])

In [25]:
result = prompt.invoke({"language" : "japanese","text":"Good morning?"})

In [27]:
result.to_messages()

[SystemMessage(content='Translate the following into japanese:', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='Good morning?', additional_kwargs={}, response_metadata={})]

In [28]:
chain = prompt | model | parser
chain.invoke({"language" : "japanese","text":"Good morning?"})

'おはようございます (Ohayou gozaimasu) \n\n\nThis is the most common way to say "Good morning" in Japanese. \n'